This jupyter notebook contains python scripting used in ArcGIS Pro to prioritize budget needs for water system improvements
Inputs include the 
1. a personal geodatabase copy of the water network pipe feature class for size, material, install date, wrapping,
2. a personal geodatabase copy of the main break data feature class for current existing pipes, without breaks on pipes that have been replaced
3. reference data includes roads, soils, could also include bridges, culverts, hydraulic model, planning & zoning, etc

Pipe categories are statistically evaluated to determine a likelihood of break ranking based on the input factors

Those rank scores are then adjusted to consider actual existing broken and repaired locations that would benefit from replacement.

Replacement costs can be compared with a statistically ranked value based on the quality of improvement to get the most benefit per dollar in scheduling routine maintenance of the system by replacing smaller, older, and failure prone pipes.

Based on the score a project year and rough cost estimates can be assigned for the CIP.

The maps and graphs are included in a related arcgis pro project that is also the default workspace for the following python script.

https://www.waterrf.org/system/files/resource/2019-05/4332_1.pdf



created by Kyle G on 7/16/2021

In [8]:
collectorwatermains = r'https://gis2.lawrenceks.org/arcgis/rest/services/Utilities_Dept/WaterCollector/FeatureServer/11'

In [9]:
arcpy.env.overwriteOutput = 1

In [10]:
#this cell will set variables referred to throughout the rest of this script for asset management calculations
#these layers without a path should exist in the active pro map
#as of about Late october, there is now a service available for breaks and leaks

watermains = "wMain221012"
#BreakData = r"C:\Users\kgonterwitz\OneDrive - City of Lawrence KS\Documents\ArcGIS\Projects\WaterAssets\WaterAssets.gdb\Main_breaks_Not_Repaired_2021_5"

BreakData = r'https://gisweb.lawrenceks.org/hosting/rest/services/Water/Water_Breaks_and_Leaks/FeatureServer'

#as of 10/13/2022 the brak data above is invalid and the endpoint, recently added, below will be used:
#this break data (below) includes transmission main points
BreakData = r'https://gisweb.lawrenceks.org/hosting/rest/services/Water/Water_Distribution_Breaks_and_Leaks/FeatureServer'

#for the map analysis add the features from above and remove them from the group layer, so the layer name in TOC is this:
BreakData = "Water Main Breaks"

Soils= r"C:\Users\kgonterwitz\OneDrive - City of Lawrence KS\Documents\ArcGIS\Projects\WaterAssets\WaterAssets.gdb\Soils_SSURGO"



In [11]:
#initiate new analysis geodatabase and copy latest updated version of water main data
projdb = r"\\citydata\users\kgonterwitz\kyledev\Waterassets20221012.gdb"



In [15]:
#Export Water Mains from hosted feature service layer to analysis geodatabase
#this will also reproject the analysis data to KRCS Zone 11

assetwhereclause = "WATERTYPE IN ('Potable') And OPERATIONAREA NOT IN ('HINU', 'KU', 'OTH', 'UNK', 'WTP') And DIAMETER <> 0"
wmains = ("https://gis2.lawrenceks.org/arcgis/rest/services/Utilities_Dept/WaterCollector/FeatureServer/11")
Zone11 = 'PROJCS["NAD_1983_2011_KS_RCS_Zone_11",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",11500000.0],PARAMETER["False_Northing",600000.0],PARAMETER["Central_Meridian",-95.25],PARAMETER["Standard_Parallel_1",39.1],PARAMETER["Scale_Factor",1.000033],PARAMETER["Latitude_Of_Origin",39.1],UNIT["Foot_US",0.3048006096012192]]'
with arcpy.EnvManager(scratchWorkspace=projdb, outputCoordinateSystem=Zone11, workspace=projdb):
    arcpy.conversion.FeatureClassToFeatureClass(wmains, projdb, watermains, assetwhereclause, '#', '')

In [5]:
#need to correct a few material types

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "MATERIAL = ' di'", None)
arcpy.management.CalculateField(watermains, "MATERIAL", '"DI"', "PYTHON3", '', "TEXT")

print(str(arcpy.management.GetCount(watermains)) + " DI pipes Corrected")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "MATERIAL = ' pvc'", None)
arcpy.management.CalculateField(watermains, "MATERIAL", '"PVC"', "PYTHON3", '', "TEXT")

print(str(arcpy.management.GetCount(watermains)) + " PVC pipes Corrected")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "MATERIAL LIKE '%sdr%'", None)
arcpy.management.CalculateField(watermains, "MATERIAL", '"PVC"', "PYTHON3", '', "TEXT")
print(str(arcpy.management.GetCount(watermains)) + " PVC pipes Corrected type 2")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION")


2 DI pipes Corrected
7 PVC pipes Corrected
1 PVC pipes Corrected type 2


<Result 'wMain221012'>

In [6]:
#create a truly unique pipe identifier

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION")
arcpy.management.AddField(watermains, "Unique_AssetID", "TEXT", None, None, 50, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(watermains, "Unique_AssetID", 'str(!FACILITYID!) +"-"+ str(!OBJECTID!)', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.AddIndex(watermains, "Unique_AssetID", "AssetID_U2", "UNIQUE", "ASCENDING")



<Result 'wMain221012'>

In [7]:
#classify pipe diameters for pipes 12" and less, including greater than 12"
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "DIAMETER > 12", None)
arcpy.management.CalculateField(watermains, "PipeDiameterScore", "6", "PYTHON3", '', "LONG")
print(str(arcpy.management.GetCount(watermains)) + " Greater Than 12 in")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "DIAMETER = 12", None)
arcpy.management.CalculateField(watermains, "PipeDiameterScore", "5", "PYTHON3", '', "LONG")
print(str(arcpy.management.GetCount(watermains)) + " 12 in")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "DIAMETER = 10", None)
arcpy.management.CalculateField(watermains, "PipeDiameterScore", "4", "PYTHON3", '', "LONG")
print(str(arcpy.management.GetCount(watermains)) + " 10 in")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "DIAMETER = 8", None)
arcpy.management.CalculateField(watermains, "PipeDiameterScore", "3", "PYTHON3", '', "LONG")
print(str(arcpy.management.GetCount(watermains)) + " 8 in")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "DIAMETER = 6", None)
arcpy.management.CalculateField(watermains, "PipeDiameterScore", "2", "PYTHON3", '', "LONG")
print(str(arcpy.management.GetCount(watermains)) + " 6 in")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "DIAMETER = 4", None)
arcpy.management.CalculateField(watermains, "PipeDiameterScore", "1", "PYTHON3", '', "LONG")
print(str(arcpy.management.GetCount(watermains)) + " 4 in")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore is null", None)
arcpy.management.CalculateField(watermains, "PipeDiameterScore", "0", "PYTHON3", '', "LONG")
print(str(arcpy.management.GetCount(watermains)) + " other diameter")

359 Greater Than 12 in
1112 12 in
62 10 in
3529 8 in
1212 6 in
231 4 in
686 other diameter


In [10]:
#Calculte Life Expectancy based on material and pipe diameter classification
arcpy.management.AddField(watermains, "ExpectedLife", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 0 And MATERIAL = 'CAS'", None)
arcpy.management.CalculateField(watermains, "ExpectedLife", "100", "PYTHON3", '', "TEXT")

print(str(arcpy.management.GetCount(watermains)) + " 0 in Cast Iron - 100 yr")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 1 And MATERIAL = 'CAS'", None)
print(str(arcpy.management.GetCount(watermains)) + " 4 in Cast Iron - 95 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "95", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 2 And MATERIAL = 'CAS'", None)
print(str(arcpy.management.GetCount(watermains)) + " 6 in Cast Iron - 85 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "95", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 3 And MATERIAL = 'CAS'", None)
print(str(arcpy.management.GetCount(watermains)) + " 8 in Cast Iron - 90 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "90", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 4 And MATERIAL = 'CAS'", None)
print(str(arcpy.management.GetCount(watermains)) + " 10 in Cast Iron - 80 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "80", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 5 And MATERIAL = 'CAS'", None)
print(str(arcpy.management.GetCount(watermains)) + " 12 in Cast Iron - 75 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "75", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 6 And MATERIAL = 'CAS'", None)
print(str(arcpy.management.GetCount(watermains)) + " Large Cast Iron - 70 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "70", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 6 And MATERIAL = 'RCP'", None)
print(str(arcpy.management.GetCount(watermains)) + " Large RCP - 100 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "100", "PYTHON3", '', "TEXT")


arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 6 And MATERIAL = 'SP'", None)
print(str(arcpy.management.GetCount(watermains)) + " Large Steel - 70 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "70", "PYTHON3", '', "TEXT")


arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 0 And MATERIAL = 'COP'", None)
print(str(arcpy.management.GetCount(watermains)) + " 0 in Copper - 70 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "70", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 3 And MATERIAL = 'COP'", None)
print(str(arcpy.management.GetCount(watermains)) + " 6 in Copper - 60 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "60", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 5 And MATERIAL = 'COP'", None)
print(str(arcpy.management.GetCount(watermains)) + " 12 in Copper - 50 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "50", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 0 And MATERIAL = 'DIP' AND EXTCOVERING is null", None)
print(str(arcpy.management.GetCount(watermains)) + " 0 in DI - 60 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "60", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 1 And MATERIAL = 'DIP'AND EXTCOVERING is null", None)
print(str(arcpy.management.GetCount(watermains)) + " 4 in DI - 55 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "55", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 2 And MATERIAL = 'DIP'AND EXTCOVERING is null", None)
print(str(arcpy.management.GetCount(watermains)) + " 6 in DI - 50- yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "50", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 3 And MATERIAL = 'DIP'AND EXTCOVERING is null", None)
print(str(arcpy.management.GetCount(watermains)) + " 8 in DI - 40 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "40", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 4 And MATERIAL = 'DIP'AND EXTCOVERING is null", None)
print(str(arcpy.management.GetCount(watermains)) + " 10 in DI - 30 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "30", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 5 And MATERIAL = 'DIP'AND EXTCOVERING is null", None)
print(str(arcpy.management.GetCount(watermains)) + " 12 in DI - 20 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "20", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 6 And MATERIAL = 'DIP'AND EXTCOVERING is null", None)
print(str(arcpy.management.GetCount(watermains)) + " Large DI - 20 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "20", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 0 And MATERIAL = 'DIP' AND EXTCOVERING is not null", None)
print(str(arcpy.management.GetCount(watermains)) + " 0 in wrapped DI poly- 80 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "80", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 1 And MATERIAL = 'DIP'AND EXTCOVERING is not null", None)
print(str(arcpy.management.GetCount(watermains)) + " 4 in wrapped DI poly- 80 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "80", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 2 And MATERIAL = 'DIP'AND EXTCOVERING is not null", None)
print(str(arcpy.management.GetCount(watermains)) + " 6 in DI poly- 75 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "75", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 3 And MATERIAL = 'DIP'AND EXTCOVERING is not null", None)
print(str(arcpy.management.GetCount(watermains)) + " 8 in DI wrapped poly- 70 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "70", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 4 And MATERIAL = 'DIP'AND EXTCOVERING is not null", None)
print(str(arcpy.management.GetCount(watermains)) + " 10 in DI wrapped poly - 60 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "60", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 5 And MATERIAL = 'DIP'AND EXTCOVERING is not null", None)
print(str(arcpy.management.GetCount(watermains)) + " 12 in wrapped DI poly - 50 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "50", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 6 And MATERIAL = 'DIP'AND EXTCOVERING is not null", None)
print(str(arcpy.management.GetCount(watermains)) + " Large wrapped DI poly - 50 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "50", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 0 And MATERIAL = 'PVC'", None)
print(str(arcpy.management.GetCount(watermains)) + " 0 in PVC - 90 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "90", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 1 And MATERIAL = 'PVC'", None)
print(str(arcpy.management.GetCount(watermains)) + " 4 in PVC - 85 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "85", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 2 And MATERIAL = 'PVC'", None)
print(str(arcpy.management.GetCount(watermains)) + " 6 in PVC - 80 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "80", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 3 And MATERIAL = 'PVC'", None)
print(str(arcpy.management.GetCount(watermains)) + " 8 in PVC - 70 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "70", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 4 And MATERIAL = 'PVC'", None)
print(str(arcpy.management.GetCount(watermains)) + " 10 in PVC - 65 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "65", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 5 And MATERIAL = 'PVC'", None)
print(str(arcpy.management.GetCount(watermains)) + " 12 in PVC - 60 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "60", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 6 And MATERIAL = 'PVC'", None)
print(str(arcpy.management.GetCount(watermains)) + " 12 in PVC - 55 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "55", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 1 And MATERIAL = 'TTE'", None)
print(str(arcpy.management.GetCount(watermains)) + " 4 in TTE - 80 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "80", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 2 And MATERIAL = 'TTE'", None)
print(str(arcpy.management.GetCount(watermains)) + " 6 in TTE - 75 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "75", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 3 And MATERIAL = 'TTE'", None)
print(str(arcpy.management.GetCount(watermains)) + " 8 in TTE - 70 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "70", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 4 And MATERIAL = 'TTE'", None)
print(str(arcpy.management.GetCount(watermains)) + " 10 in TTE - 60 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "60", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 5 And MATERIAL = 'TTE'", None)
print(str(arcpy.management.GetCount(watermains)) + " 12 in TTE - 50 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "50", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "PipeDiameterScore = 6 And MATERIAL = 'TTE'", None)
print(str(arcpy.management.GetCount(watermains)) + " Large TTE - 45 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "45", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "ExpectedLife is null AND WATERTYPE = 'Potable'", None)
print(str(arcpy.management.GetCount(watermains)) + " unknown combo- 33 yr")
arcpy.management.CalculateField(watermains, "ExpectedLife", "33", "PYTHON3", '', "TEXT")

14 0 in Cast Iron - 100 yr
52 4 in Cast Iron - 95 yr
368 6 in Cast Iron - 85 yr
437 8 in Cast Iron - 90 yr
35 10 in Cast Iron - 80 yr
166 12 in Cast Iron - 75 yr
87 Large Cast Iron - 70 yr
111 Large RCP - 100 yr
11 Large Steel - 70 yr
352 0 in Copper - 70 yr
0 6 in Copper - 60 yr
1 12 in Copper - 50 yr
3 0 in DI - 60 yr
14 4 in DI - 55 yr
130 6 in DI - 50- yr
435 8 in DI - 40 yr
9 10 in DI - 30 yr
424 12 in DI - 20 yr
96 Large DI - 20 yr
0 0 in wrapped DI poly- 80 yr
2 4 in wrapped DI poly- 80 yr
5 6 in DI poly- 75 yr
16 8 in DI wrapped poly- 70 yr
1 10 in DI wrapped poly - 60 yr
43 12 in wrapped DI poly - 50 yr
21 Large wrapped DI poly - 50 yr
197 0 in PVC - 90 yr
146 4 in PVC - 85 yr
680 6 in PVC - 80 yr
2626 8 in PVC - 70 yr
17 10 in PVC - 65 yr
469 12 in PVC - 60 yr
26 12 in PVC - 55 yr
13 4 in TTE - 80 yr
26 6 in TTE - 75 yr
8 8 in TTE - 70 yr
0 10 in TTE - 60 yr
5 12 in TTE - 50 yr
3 Large TTE - 45 yr
134 unknown combo- 33 yr


<Result 'wMain221012'>

In [8]:
#evaluate install dates that are null, there are about 17 records, most of them are 8" pvc
#some might be recent edits 
#most are non-city infrastructure, like KU or HINU
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE IS NULL", None)
arcpy.management.CalculateField(watermains, "INSTALLDATE", '"1/1/1999"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")



<Result 'wMain221012'>

In [11]:
#determine remaining life in years 
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION")

arcpy.management.AddField(watermains, "LifeRemaining", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "ExpectedLife IS NOT NULL And INSTALLDATE IS NOT NULL", None)
arcpy.management.CalculateField(watermains, "LifeRemaining", "!ExpectedLife!-(2022-int(str(!INSTALLDATE!)[:4]))", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "LifeRemaining < 1 OR LifeRemaining is null", None)
arcpy.management.CalculateField(watermains, "LifeRemaining", "0", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION")

<Result 'wMain221012'>

In [13]:
#calculate the percent of remaining life left for each pipe segment in GIS

arcpy.management.AddField(watermains, "PercentLifeRemaining", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
#arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "LifeRemaining > 0", None)
arcpy.management.CalculateField(watermains, "PercentLifeRemaining", "!LifeRemaining!/!ExpectedLife! * 100", "PYTHON3", '', "TEXT")



<Result 'wMain221012'>

In [16]:
#add a field converting shape length feet to miles for total summary

arcpy.management.CalculateField(watermains, "MilesOfPipe", "!Shape_Length!/5280", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

<Result 'wMain221012'>

In [17]:
#exploring pandas data frames, pivot tables, and grouping material and diameter summary lengths
#this result would be useful for data quality control 


import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

#look at total pipe length in feet by material with 10 or less years life remaining

#arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "LifeRemaining <11", None)

#Look at all values to check 
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION")

fc = watermains

fields = ['MATERIAL','MilesOfPipe', 'DIAMETER'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = pd.pivot_table(df, values='MilesOfPipe', index=['MATERIAL'], columns=['DIAMETER'], aggfunc=sum, margins = True)

#print(table)
#display(table)

from IPython.display import display, HTML

display(HTML(table.to_html()))

DIAMETER,0.0,0.75,1.0,1.25,1.5,2.0,2.5,3.0,4.0,6.0,8.0,10.0,12.0,14.0,15.0,16.0,18.0,20.0,24.0,30.0,36.0,42.0,All
MATERIAL,,,,,,,,,,,,,,,,,,,,,,,
CAS,NaN,NaN,NaN,NaN,NaN,0.072757,NaN,0.128160,1.657342,25.335600,29.273271,1.799802,13.009178,1.025765,0.165485,2.854591,0.145612,NaN,0.119795,0.244253,NaN,NaN,75.831612
COP,NaN,0.044987,0.713202,NaN,0.053292,9.370184,0.027587,0.006188,0.047965,0.036696,NaN,NaN,0.000139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.300239
DI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005186
DIP,NaN,NaN,NaN,NaN,NaN,0.001996,NaN,0.003547,0.253042,7.642456,27.703820,0.184468,38.030006,0.030507,NaN,8.230892,NaN,0.188899,4.847223,0.048588,0.095054,NaN,87.260499
GP,NaN,NaN,NaN,NaN,NaN,0.183145,NaN,NaN,0.000165,0.002332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.185641
NIL,NaN,NaN,NaN,NaN,NaN,0.447419,NaN,NaN,0.001162,NaN,0.241474,NaN,NaN,NaN,NaN,0.038498,NaN,NaN,0.052541,NaN,NaN,NaN,0.781095
PE,NaN,NaN,0.053265,NaN,NaN,1.507587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.560851
PVC,0.003606,0.105963,0.130805,0.05591,0.064556,5.629453,NaN,0.896018,6.172627,32.507393,186.311620,0.305387,37.247311,0.008903,NaN,2.242961,NaN,0.118836,NaN,0.022878,3.029052,NaN,274.853277
RCP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000447,NaN,6.093417,NaN,0.574800,3.854192,0.004940,0.114896,0.113895,10.756587


In [18]:
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION")

<Result 'wMain221012'>

In [19]:
#calculate the LOB Area statistics - consider pipes by material, coverings, and install period in decades
#calculate the install era - decade

arcpy.management.AddField(watermains, "Install_ERA", "TEXT", None, None, 12, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1900-01-01 00:00:00'", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1850-1899"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1910-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1900-1909"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1920-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1910-1919"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1930-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1920-1929"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1940-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1930-1939"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1950-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1940-1949"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1960-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1950-1959"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1970-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1960-1969"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1980-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1970-1979"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '1990-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1980-1989"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '2000-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"1990-1999"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '2010-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"2000-2009"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '2020-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"2010-2019"', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "INSTALLDATE < timestamp '2030-01-01 00:00:00' And Install_ERA IS NULL", None)
arcpy.management.CalculateField(watermains, "Install_ERA", '"2020-2029"', "PYTHON3", '', "TEXT")


arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION")


<Result 'wMain221012'>

In [20]:
#pivot table grouping material and diameter summary lengths
#data quality control review


import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable'", None)

fc = watermains

fields = ['MATERIAL', 'Install_Era', 'MilesOfPipe'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)

table = pd.pivot_table(df, values='MilesOfPipe', index=['MATERIAL'], columns=['Install_Era'], aggfunc=sum, margins = True, dropna = True)
#print(table)
#display(table)

from IPython.display import display, HTML

display(HTML(table.to_html()))

Install_Era,1850-1899,1900-1909,1910-1919,1920-1929,1930-1939,1940-1949,1950-1959,1960-1969,1970-1979,1980-1989,1990-1999,2000-2009,2010-2019,2020-2029,All
MATERIAL,,,,,,,,,,,,,,,
CAS,0.376638,0.065132,1.165686,2.026790,0.834555,0.619386,10.523989,34.796236,20.680632,0.811577,1.669883,0.012914,0.016682,NaN,73.600099
COP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.231639,1.224940,2.012057,3.812975,2.945938,0.072689,NaN,10.300239
DI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005186,NaN,NaN,NaN,0.005186
DIP,NaN,NaN,NaN,NaN,NaN,NaN,0.503390,1.575452,11.894809,25.039556,17.960493,24.956530,4.427502,0.011027,86.368760
GP,NaN,NaN,0.000165,0.002269,NaN,0.000395,0.099780,0.080701,NaN,NaN,NaN,NaN,NaN,NaN,0.183310
NIL,NaN,NaN,NaN,NaN,NaN,NaN,0.001162,NaN,0.241474,NaN,0.002814,0.272973,0.171632,NaN,0.690055
PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.101414,1.352244,0.107194,1.560851
PVC,NaN,0.004536,0.016874,0.070868,NaN,0.001123,0.270051,0.495157,13.934921,34.533877,90.688166,59.121094,65.982021,3.581214,268.699903
RCP,NaN,NaN,NaN,NaN,NaN,NaN,6.614591,NaN,3.556114,0.344724,NaN,0.000447,NaN,NaN,10.515875


In [21]:
#pivot table grouping material and diameter summary lengths
#data quality control 


import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable'", None)

fc = watermains

fields = ['MATERIAL', 'Install_Era', 'MilesOfPipe'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)

table = pd.pivot_table(df, values='MilesOfPipe', index=['MATERIAL'], columns=['Install_Era'], aggfunc=sum, margins = True, dropna = True)
#print(table)
#display(table)

from IPython.display import display, HTML

display(HTML(table.to_html()))

Install_Era,1850-1899,1900-1909,1910-1919,1920-1929,1930-1939,1940-1949,1950-1959,1960-1969,1970-1979,1980-1989,1990-1999,2000-2009,2010-2019,2020-2029,All
MATERIAL,,,,,,,,,,,,,,,
CAS,0.376638,0.065132,1.165686,2.026790,0.834555,0.619386,10.523989,34.796236,20.680632,0.811577,1.669883,0.012914,0.016682,NaN,73.600099
COP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.231639,1.224940,2.012057,3.812975,2.945938,0.072689,NaN,10.300239
DI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005186,NaN,NaN,NaN,0.005186
DIP,NaN,NaN,NaN,NaN,NaN,NaN,0.503390,1.575452,11.894809,25.039556,17.960493,24.956530,4.427502,0.011027,86.368760
GP,NaN,NaN,0.000165,0.002269,NaN,0.000395,0.099780,0.080701,NaN,NaN,NaN,NaN,NaN,NaN,0.183310
NIL,NaN,NaN,NaN,NaN,NaN,NaN,0.001162,NaN,0.241474,NaN,0.002814,0.272973,0.171632,NaN,0.690055
PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.101414,1.352244,0.107194,1.560851
PVC,NaN,0.004536,0.016874,0.070868,NaN,0.001123,0.270051,0.495157,13.934921,34.533877,90.688166,59.121094,65.982021,3.581214,268.699903
RCP,NaN,NaN,NaN,NaN,NaN,NaN,6.614591,NaN,3.556114,0.344724,NaN,0.000447,NaN,NaN,10.515875


In [27]:
#score the pipes based on corrosive soils - add the soils layer to the analysis map
#soils dont appear to contribute to higher LOB, we should stop doing this.  
#it can still be added to continue to test sensitivity to corrosive soils but dont use this grouping for LOB
#The SSURGO Soils layer has been added to the portal, it has to be added to the map named Soils_SSURGO, or I can change this code to use the feature service

soils = r'https://gisweb.lawrenceks.org/hosting/rest/services/Hosted/Douglas_County_Soils/FeatureServer'

arcpy.management.AddField(watermains, "SoilCorrosionValue", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

arcpy.management.SelectLayerByAttribute("Soils_SSURGO", "NEW_SELECTION", "steelcorr = 1", None)
arcpy.management.SelectLayerByLocation(watermains, "HAVE_THEIR_CENTER_IN", "Soils_SSURGO", None, "NEW_SELECTION", "NOT_INVERT")

arcpy.management.SelectLayerByAttribute("Soils_SSURGO", "NEW_SELECTION", "steelcorr = 2", None)
arcpy.management.SelectLayerByLocation(watermains, "HAVE_THEIR_CENTER_IN", "Soils_SSURGO", None, "NEW_SELECTION", "NOT_INVERT")
arcpy.management.CalculateField(watermains, "SoilCorrosionValue", "3", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute("Soils_SSURGO", "NEW_SELECTION", "steelcorr = 3", None)
arcpy.management.SelectLayerByLocation(watermains, "HAVE_THEIR_CENTER_IN", "Soils_SSURGO", None, "NEW_SELECTION", "NOT_INVERT")
arcpy.management.CalculateField(watermains, "SoilCorrosionValue", "5", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "SoilCorrosionValue is null", None)
arcpy.management.CalculateField(watermains, "SoilCorrosionValue", "0", "PYTHON3", '', "TEXT")


arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)
arcpy.management.SelectLayerByAttribute("Soils_SSURGO", "CLEAR_SELECTION", '', None)

<Result 'Soils_SSURGO'>

In [28]:
#Score the pipes based on the proximity to Highway Functional Classification
#these values exist in the excel table but do not seem to be used in the analysis and are not referenced in the report
#this is used to heighten risk of breaks for pipes in the right of way and impacts to traffic and pavement

roadcenterlines = r'https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/RoadCenterline/FeatureServer'

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)
arcpy.management.SelectLayerByAttribute("Road Centerline", "CLEAR_SELECTION", '', None)

arcpy.management.AddField(watermains, "StreetClassValue", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute("Road Centerline", "NEW_SELECTION", "FUNCTCLASS = 'PRINCIPAL ARTERIAL'", None)
arcpy.management.SelectLayerByLocation(watermains, "WITHIN_A_DISTANCE", "Road Centerline", "60 Feet", "NEW_SELECTION", "NOT_INVERT")
arcpy.management.CalculateField(watermains, "StreetClassValue", "5", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute("Road Centerline", "NEW_SELECTION", "FUNCTCLASS in ('MINOR ARTERIAL', 'MAJOR COLLECTOR')", None)
arcpy.management.SelectLayerByLocation(watermains, "WITHIN_A_DISTANCE", "Road Centerline", "40 Feet", "NEW_SELECTION", "NOT_INVERT")
arcpy.management.SelectLayerByAttribute(watermains, "REMOVE_FROM_SELECTION", "StreetClassValue IS NOT NULL", None)
arcpy.management.CalculateField(watermains, "StreetClassValue", "4", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute("Road Centerline", "NEW_SELECTION", "FUNCTCLASS in ('COLLECTOR', 'MINOR COLLECTOR')", None)
arcpy.management.SelectLayerByLocation(watermains, "WITHIN_A_DISTANCE", "Road Centerline", "40 Feet", "NEW_SELECTION", "NOT_INVERT")
arcpy.management.SelectLayerByAttribute(watermains, "REMOVE_FROM_SELECTION", "StreetClassValue IS NOT NULL", None)
arcpy.management.CalculateField(watermains, "StreetClassValue", "3", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute("Road Centerline", "NEW_SELECTION", "FUNCTCLASS in ('STREET', 'OTHER RAMP')", None)
arcpy.management.SelectLayerByLocation(watermains, "WITHIN_A_DISTANCE", "Road Centerline", "40 Feet", "NEW_SELECTION", "NOT_INVERT")
arcpy.management.SelectLayerByAttribute(watermains, "REMOVE_FROM_SELECTION", "StreetClassValue IS NOT NULL", None)
arcpy.management.CalculateField(watermains, "StreetClassValue", "2", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute("Road Centerline", "NEW_SELECTION", "FUNCTCLASS in ('PRIVATE', 'ALLEY')", None)
arcpy.management.SelectLayerByLocation(watermains, "WITHIN_A_DISTANCE", "Road Centerline", "40 Feet", "NEW_SELECTION", "NOT_INVERT")
arcpy.management.SelectLayerByAttribute(watermains, "REMOVE_FROM_SELECTION", "StreetClassValue IS NOT NULL", None)
arcpy.management.CalculateField(watermains, "StreetClassValue", "1", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "StreetClassValue IS NULL", None)
arcpy.management.CalculateField(watermains, "StreetClassValue", "0", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)
arcpy.management.SelectLayerByAttribute("Road Centerline", "CLEAR_SELECTION", '', None)

<Result 'Road Centerline'>

In [8]:
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION")

<Result 'Water_Main_2022_05_26'>

In [22]:
#Assetic Life Variation and LOB variaion Concatenations SANS Corrosion values
#not very many records have a polywrap or any type of external covering value

arcpy.management.AddField(watermains, "LOB_Variation", "TEXT", None, None, 26, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(watermains, "LOB_Variation", 'str(!MATERIAL!)+"-"+str(int(!DIAMETER!))+"-"+str(!Install_ERA!)', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "EXTCOVERING IS NOT NULL", None)
arcpy.management.CalculateField(watermains, "LOB_Variation", 'str(!MATERIAL!)+"-Poly-"+str(int(!DIAMETER!))+"-"+str(!Install_ERA!)', "PYTHON3", '', "TEXT")

#arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "LOB_Variation IS NULL", None)
#arcpy.management.CalculateField(watermains, "LOB_Variation", 'str(!MATERIAL!)+"-Poly-"+str(int(!DIAMETER!))+"-?-"+str(!SoilCorrosionValue!)', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)



<Result 'wMain221012'>

In [6]:
#Assetic Life Variation and LOB variaion Concatenations with soil corrosion included - dont do this any more
#not very many records have a polywrap or any type of external covering value

'''
arcpy.management.AddField(watermains, "LOB_Variation", "TEXT", None, None, 26, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(watermains, "LOB_Variation", 'str(!MATERIAL!)+"-"+str(int(!DIAMETER!))+"-"+str(!Install_ERA!)+"-"+str(!SoilCorrosionValue!)', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "EXTCOVERING IS NOT NULL", None)
arcpy.management.CalculateField(watermains, "LOB_Variation", 'str(!MATERIAL!)+"-Poly-"+str(int(!DIAMETER!))+"-"+str(!Install_ERA!)+"-"+str(!SoilCorrosionValue!)', "PYTHON3", '', "TEXT")

#arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "LOB_Variation IS NULL", None)
#arcpy.management.CalculateField(watermains, "LOB_Variation", 'str(!MATERIAL!)+"-Poly-"+str(int(!DIAMETER!))+"-?-"+str(!SoilCorrosionValue!)', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)

'''

id,value
0,a Layer object
1,-1


In [31]:
# can skip, but it is referenced in the assetic report

#arcpy.management.AddField(watermains, "Life_Variation", "TEXT", None, None, 16, '', "NULLABLE", "NON_REQUIRED", '')

arcpy.management.CalculateField(watermains, "Life_Variation", 'str(!MATERIAL!)+"-"+str(int(!DIAMETER!))+"-"+str(!SoilCorrosionValue!)', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "EXTCOVERING IS NOT NULL", None)
arcpy.management.CalculateField(watermains, "Life_Variation", 'str(!MATERIAL!)+"-Poly-"+str(!SoilCorrosionValue!)', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "LOB_Variation IS NULL", None)
arcpy.management.CalculateField(watermains, "Life_Variation", 'str(!MATERIAL!)+"-"+str(int(!DIAMETER!))+"-?-"+str(!SoilCorrosionValue!)', "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)


<Result 'Water_Main_2022_05_26'>

In [23]:
#join mainbreak data to the water mains
#dissolve water mains by the calculated properties summing the waterlines that have associated main break records
#this does not tally the count of breaks per main, it just flags the closest main to the main break point from lucity 
#the count of mains that have broken can be used later to adjust the LOB
#the main break data includes many appearant duplicate records that may skew the results - 
#the main breaks per 100 feet are not counting actual main breaks, but for predicting performnance of similar pipe diameters, materials, and ages pipes for breakage statistics.  
#If one pipe has had about 5 or more repairs that not typical, for our system, of all the similar pipes of that same diameter, material, and install era.
#that pipe with 5 repairs probably needs replaced though, and that will be accounted for in a LOB_ADJ and will rank high in the final prioritization
#break data changed a little bit

BreakData = "Water Main Breaks"

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)

arcpy.analysis.SpatialJoin(watermains, BreakData, projdb+r"/"+watermains+"_BreakJoin", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "CLOSEST", "100 Feet", "BreakDataDistance")

arcpy.management.Dissolve(watermains+"_BreakJoin", projdb+r"/"+watermains+"_Break_Stats", "Install_ERA;DIAMETER;MATERIAL;LOB_Variation", "Join_Count SUM;OBJECTID COUNT", "MULTI_PART", "DISSOLVE_LINES")

arcpy.analysis.Statistics(watermains+"_Break_Stats", projdb+r"/"+watermains+"_Check_Breaks", "SUM_Join_Count SUM", None)


<Result '\\\\citydata\\users\\kgonterwitz\\kyledev\\Waterassets20221012.gdb\\wMain221012_Check_Breaks'>

In [26]:
#review breaks with newer mains than the break date
#Break data fields changed a bit including this one 10/14

#arcpy.management.SelectLayerByAttribute(watermains+"_BreakJoin", "NEW_SELECTION", "INSTALLDATE >= BreakReportedDate", None)
arcpy.management.SelectLayerByAttribute(watermains+"_BreakJoin", "NEW_SELECTION", "INSTALLDATE >= WOStartDate", None)


<Result 'wMain221012_BreakJoin'>

In [26]:
#review breaks with newer mains than the break date
#with DELETE of those selected rows - after reviewed

#arcpy.management.SelectLayerByAttribute(watermains+"_BreakJoin", "NEW_SELECTION", "INSTALLDATE >= BreakReportedDate", None)
arcpy.management.SelectLayerByAttribute(watermains+"_BreakJoin", "NEW_SELECTION", "INSTALLDATE >= WOStartDate", None)
arcpy.management.DeleteRows(watermains+"_BreakJoin")

<Result 'wMain221012_BreakJoin'>

In [9]:
#select valid breaks, not breaks on mains that have been replaced

arcpy.management.SelectLayerByAttribute("Water_Main_2022_05_26_BreakJoin", "NEW_SELECTION", "INSTALLDATE <= BreakReportedDate", None)

#review and delete records from Breakjoin table

<Result 'Water_Main_2022_05_26_BreakJoin'>

In [27]:
#Dissolve fields changed 10/14
arcpy.management.Dissolve(watermains+"_BreakJoin", projdb+r"/"+watermains+"_Break_Stats", "Install_ERA;DIAMETER;MATERIAL;LOB_Variation", "Join_Count SUM;OBJECTID COUNT", "MULTI_PART", "DISSOLVE_LINES")
arcpy.analysis.Statistics(watermains+"_Break_Stats", projdb+r"/"+watermains+"_Check_Breaks", "SUM_Join_Count SUM", None)


<Result '\\\\citydata\\users\\kgonterwitz\\kyledev\\Waterassets20221012.gdb\\wMain221012_Check_Breaks'>

In [28]:
arcpy.analysis.Statistics(watermains+"_Break_Stats", projdb+r"/"+watermains+"_Check_Length", "Shape_Length SUM", None)
arcpy.management.CalculateField(watermains+"_Check_Length", "TotalLengthMiles", "!SUM_Shape_Length!/5280", "PYTHON3", '', "Double")

<Result 'wMain221012_Check_Length'>

In [29]:
#calculate breaks per 100 feet
#select segments that are less than 400 feet and set the breaks to an "average" so short lengths/small sample combinations arent overweight

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "BreaksPer100ft", "!SUM_Join_Count!/(!Shape_Length!/100)", "PYTHON3", '', "DOUBLE")
arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "Shape_Length < 400", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "BreaksPer100ft", "0.1", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "CLEAR_SELECTION", '', None)


<Result 'wMain221012_Break_Stats'>

In [30]:
#calculate likelihood of breaks integer value ranges
#a number of ways exist to do this, 
#use the symbology mapping in ArcPro Mapping can help statiscically evaluate the range values to use in a given analysis period and budget forecast
#The values here use are based on the jenks breaks rounded nicely
#these break values are similar to the predictor prep excel sheet
#with this code each pipe that has breaks has one break counted as noted above
#it is not weighted more heavily by individual pipes with multiple breaks - that is done on the LOB ADJ field

arcpy.management.AddField(watermains+"_Break_Stats", "LOB", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft < 0.015", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "0", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.015 AND BreaksPer100ft < 0.05", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "1", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.05 AND BreaksPer100ft < 0.1", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "2", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.1 AND BreaksPer100ft < 0.15", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "3", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.15 AND BreaksPer100ft < 0.2", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "4", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.2 AND BreaksPer100ft <= 0.5", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "5", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >0.5", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "6", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "CLEAR_SELECTION", '', None)



<Result 'wMain221012_Break_Stats'>

In [9]:
#calculate likelihood of breaks integer value ranges
#a number of ways exist to do this, 
#use the symbology mapping in ArcPro Mapping can help statiscically evaluate the range values to use in a given analysis period and budget forecast
#The values here use are based on the jenks breaks rounded nicely
#these break values have been adjusted based on materials corrections
#with this code each pipe that has breaks has one break counted as noted above
#it is not weighted more heavily by individual pipes with multiple breaks - that is done on the LOB ADJ field

#arcpy.management.AddField("Water_Main_Break_Stats", "LOB", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft < 0.0125", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "0", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.0125 AND BreaksPer100ft < 0.04", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "1", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.04 AND BreaksPer100ft < 0.06", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "2", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.06 AND BreaksPer100ft < 0.09", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "3", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.09 AND BreaksPer100ft < 0.15", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "4", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >= 0.15 AND BreaksPer100ft <= 0.5", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "5", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "NEW_SELECTION", "BreaksPer100ft >0.5", None)
arcpy.management.CalculateField(watermains+"_Break_Stats", "LOB", "6", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "CLEAR_SELECTION", '', None)



<Result 'Water_Main_2022_05_26_Break_Stats'>

In [10]:
#add an attribute index to LOB_Variation
arcpy.management.AddIndex(watermains+"_Break_Stats", "LOB_Variation", "statLOB", "NON_UNIQUE", "NON_ASCENDING")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000192: Invalid value for Index Name
Failed to execute (AddIndex).


In [6]:
#add an attribute index to LOB_Variation
arcpy.management.AddIndex(watermains, "LOB_Variation", "mainLOB", "NON_UNIQUE", "NON_ASCENDING")

<Result 'wMain221012'>

In [4]:
#the likelihood of break has been statistically set for each group of pipes,
#now the summary pipe information can be joined back to the individual pipe segments for prioritization
#the new breaks service had 240+ pipes with breakes older than the pipe, , check that selections is working here for pipe/break dates

arcpy.management.AddField(watermains, "LOB", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.AddJoin(watermains, "LOB_Variation", watermains+"_Break_Stats", "LOB_Variation", "KEEP_ALL")
arcpy.management.CalculateField(watermains, "LOB", "!"+watermains+"_Break_Stats.LOB!", "PYTHON3", '', "TEXT")
#arcpy.management.RemoveJoin(watermains, watermains+"_Break_Stats")


<Result 'wMain221012'>

In [5]:
#note 52/27/2022 there are a bunch of null LOBs at this point, missing variations
#joining problem using method, a very small number of records are joining the rest have null values
#arcgis pro suspicious behvaior on this join, exporting to sql server for evaluation
#updated project with pro v3 and now the join/calculate worked flawlessly

arcpy.management.RemoveJoin(watermains, watermains+"_Break_Stats")

<Result 'wMain221012'>

In [ ]:
#run processing in sql server
#develop views and queries that do most of this stuff
#this sql query updates the Pipe LOB from the LOB Variation break statistics


'''
 UPDATE [LAWRENCE\kgonterwitz].[WATER_MAIN_2022_05_26]
 SET [WATER_MAIN_2022_05_26].LOB = b.LOB
 FROM [LAWRENCE\kgonterwitz].[WATER_MAIN_2022_05_26] a
  left join WATER_MAIN_2022_05_31_BREAK_STATS b ON a.LOB_Variation = b.LOB_Variation

'''


In [13]:
#join main data to the closest water mains to obtain more details about the counts of breaks per water main
#the main break data contains some appearant duplicates so this needs to be re-run with recent and clean break data for accurate results
#the break results should also be filtered temporally, so that breaks dont appear to occur on pipes that have been replaced 
#add the break count per pipe to the pipe table and set the LOB Adj
#12/3/2021 field names for analysis adjusted to account for new feature service layer of breaks
#query added to remove oudated break
# note that the new break data has a field called ASSET_ID that is the non-unique facility ID
# to avoid confusion over this it the unique asset ID field has been renamed 

arcpy.management.AddField(watermains, "BreakCount", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

arcpy.analysis.SpatialJoin(BreakData, watermains, projdb+r"/"+watermains+"_Join", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "CLOSEST", "100 Feet", "NearestMainDist")
#at this point with the new feature layer the breaks data doesnt know when or if the pipe has been replaced.  

#this selection and deletion will remove those breaks from the dataset that occured before a pipe was replaced
arcpy.management.SelectLayerByAttribute(watermains+"_Join", "NEW_SELECTION", "WOStartDate < INSTALLDATE OR INSTALLDATE IS NULL" , None)
arcpy.management.DeleteFeatures(watermains+"_Join")

arcpy.management.Dissolve(watermains+"_Join", projdb+r"\BreaksPerPipe", "Unique_AssetID", "Join_Count SUM;WOStartDate MIN;WOStartDate MAX;CauseType FIRST;CauseType LAST", "MULTI_PART", "DISSOLVE_LINES")
arcpy.management.AddJoin(watermains, "Unique_AssetID", "BreaksPerPipe", "Unique_AssetID", "KEEP_ALL")
arcpy.management.CalculateField(watermains, "BreakCount", "!BreaksPerPipe.SUM_Join_Count!", "PYTHON3", '', "TEXT")

arcpy.management.RemoveJoin(watermains, "BreaksPerPipe")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount < 1 or BreakCount is null", None)
arcpy.management.CalculateField(watermains, "BreakCount", "0", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)

<Result 'wMain221012'>

In [14]:
#adjust break data for querification
#the break join now brings together the fields to compare breaks and pipes for dates, materials and diameters
#interrogate this data closer to attempt to validate the correct break data joins
#based on initial diameter analysis it was pretty good
#break data changed


arcpy.management.SelectLayerByAttribute("wMain221012_Join", "NEW_SELECTION", "DIAMETER_1 <> Diameter", None)
#181 results
print(arcpy.management.GetCount("wMain221012_Join"))

181


In [6]:
#adjust break data for querification
#the break join now brings together the fields to compare breaks and pipes for dates, materials and diameters
#interrogate this data closer to attempt to validate the correct break data joins
#based on initial diameter analysis it was pretty good
#break data changed


arcpy.management.SelectLayerByAttribute("wMain221012_Join", "NEW_SELECTION", "DIAMETER_1 <> Diameter", None)
#181 results
print(arcpy.management.GetCount("wMain221012_Join"))

181


In [16]:
#display the rows to be deleted because the reported diameters differ
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
data = "wMain221012_Join"
df = pd.DataFrame.spatial.from_featureclass(data) 
df.sort_values("NearestMainDist", inplace=True) 
df.drop(['OBJECTID', 'SHAPE'], axis=1, inplace=True)
df.style.hide_index()



Join_Count,NearestMainDist,Category,ProblemType,CauseType,ActionType,WOStartDate,WOEndDate,Diameter,Material,PressureZone,InstallDate,INSTALLDATE_1,DIAMETER_1,LABELFIELD
1,0.000000,Water Distribution Main,Leak - Water Main,Corrosion,Inspect,2022-04-11 07:00:00,2022-04-12 07:00:00,2.000000,Copper,West Hills (HP),1986-01-01 00:00:00,1969-01-01 00:00:00,6.000000,6'' ci 1969
1,0.000000,Water Distribution Main,Leak - Water Main,Corrosion,Replace Section Of Pipe,2018-05-19 07:00:00,2018-05-24 07:00:00,12.000000,Cast Iron,West Hills (HP),1967-01-01 00:00:00,1975-01-01 00:00:00,8.000000,8'' ci 1975
1,0.000122,Water Distribution Main,Leak - Water Main,Joint Failure,Replace Section Of Pipe,2018-03-13 07:00:00,2018-05-08 07:00:00,12.000000,Polyvinyl Chloride,West Hills (HP),1986-01-01 00:00:00,2006-01-01 00:00:00,8.000000,8in pvc sdr-14 c-900 2006
1,0.028500,Water Distribution Main,Leak - Water Main,Corrosion,Other,2009-01-22 07:00:00,2009-01-22 07:00:00,12.000000,Cast Iron,West Hills (HP),1954-01-01 00:00:00,1968-01-01 00:00:00,8.000000,8'' ci
1,0.077336,Water Distribution Main,Leak - Water Main,Defective Material,Repair,2009-10-22 07:00:00,2009-10-22 07:00:00,2.000000,Copper,West Hills (HP),1986-01-01 00:00:00,1986-01-01 00:00:00,6.000000,6'' pvc sdr-14 c-900 1986
1,0.120722,Water Distribution Main,Leak - Water Main,Corrosion,Install Repair Clamp,2011-09-13 07:00:00,2011-09-13 07:00:00,6.000000,Cast Iron,Central Service (LP),1955-01-01 00:00:00,2012-07-10 00:00:00,8.000000,"8"" C900 2012"
1,0.142873,Water Distribution Main,Leak - Water Main,Corrosion - Differential Settlement,Replace Section of Pipe,2017-03-29 07:00:00,2017-03-30 07:00:00,8.000000,Ductile Iron,Central Service (LP),2001-01-01 00:00:00,1996-01-01 00:00:00,6.000000,6'' pvc sdr-14 c-900 1996
1,0.154888,Water Distribution Main,Leak - Water Main,Corrosion,Install Repair Clamp,2013-12-14 07:00:00,2013-12-14 07:00:00,6.000000,Cast Iron,Central Service (LP),1954-01-01 00:00:00,2015-09-09 00:00:00,8.000000,"8"" C900 2015"
1,0.159229,Water Distribution Main,Leak - Water Main,Corrosion,Replace Section of Pipe,2011-09-14 07:00:00,2011-09-14 07:00:00,6.000000,Cast Iron,Central Service (LP),1955-01-01 00:00:00,2012-07-10 00:00:00,8.000000,"8"" C900 2012"
1,0.200761,Water Distribution Main,Leak - Water Main,Water Valve Repair,Replace Valve,2018-11-26 07:00:00,2018-11-26 07:00:00,12.000000,Cast Iron,Central Service (LP),1978-01-01 00:00:00,1991-01-01 00:00:00,4.000000,4in pvc 1991


In [17]:
arcpy.management.DeleteRows("wMain221012_Join")
#this method requires a save of hte edits

<Result 'wMain221012_Join'>

In [20]:
arcpy.env.overwriteOutput = 1

In [24]:
#Update the breaks per pipe with the better matching information based on lucity diameter not matching GIS diameter

arcpy.management.Dissolve(watermains+"_Join", projdb+r"\BreaksPerPipe", "Unique_AssetID", "Join_Count SUM;WOStartDate MIN;WOStartDate MAX;CauseType FIRST;CauseType LAST", "MULTI_PART", "DISSOLVE_LINES")
arcpy.management.AddJoin(watermains, "Unique_AssetID", "BreaksPerPipe", "Unique_AssetID", "KEEP_ALL")
arcpy.management.CalculateField(watermains, "BreakCount", "!BreaksPerPipe.SUM_Join_Count!", "PYTHON3", '', "TEXT")

arcpy.management.RemoveJoin(watermains, "BreaksPerPipe")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount < 1 or BreakCount is null", None)
arcpy.management.CalculateField(watermains, "BreakCount", "0", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)

<Result 'wMain221012'>

In [25]:
#score the LOB ADJ field

arcpy.management.AddField(watermains, "LOB_ADJ", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount < 1 or BreakCount is null", None)
arcpy.management.CalculateField(watermains, "LOB_ADJ", "0", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount = 1", None)
arcpy.management.CalculateField(watermains, "LOB_ADJ", "1", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount = 2", None)
arcpy.management.CalculateField(watermains, "LOB_ADJ", "2", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount = 3", None)
arcpy.management.CalculateField(watermains, "LOB_ADJ", "3", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount = 4", None)
arcpy.management.CalculateField(watermains, "LOB_ADJ", "4", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount = 5", None)
arcpy.management.CalculateField(watermains, "LOB_ADJ", "5", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "BreakCount > 5", None)
arcpy.management.CalculateField(watermains, "LOB_ADJ", "6", "PYTHON3", '', "TEXT")

arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)



<Result 'wMain221012'>

In [26]:
#this replacement cost per linear foot per pipe diameter was estimated using a polynomial equation based on emperical statistics graphed in excel
#the emperical data included cost up to 12" and I made up guestimate numbers for higher diameter pipes that should be verified emperically
#this should calculate the replacement value, so that we can prioritize replacement of smaller pipes with 8" and changes to improve system performance bang for the buck

DIAM = 20
costLF = 1.2*DIAM*DIAM-12*DIAM+240

print(costLF)

arcpy.management.AddField(watermains, "ReplacementValue", "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(watermains, "ReplacementValue", "(1.2*!DIAMETER!*!DIAMETER!-12*!DIAMETER!+240)*!Shape_Length!", "PYTHON3", '', "TEXT")


480.0


<Result 'wMain221012'>

In [27]:
#for replacement scheduling and budgeting, consider replacement cost- action for replacing with 8" pvc minimum pipe sizes
#for budgeting, consider comperehensive plan and transmission main extensions along arterial and collector growth corridors for larger pipe sizes

#based on cost groups and replacement needs, model the replacement schedule per year for the next 20 or 30 years

arcpy.management.AddField(watermains, "ReplacementCost", "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(watermains, "ReplacementCost", "(1.2*!DIAMETER!*!DIAMETER!-12*!DIAMETER!+240)*!Shape_Length!", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "DIAMETER < 8 AND DIAMETER > 2", None)
arcpy.management.CalculateField(watermains, "ReplacementCost", "(1.2*64-12*8+240)*!Shape_Length!", "PYTHON3", '', "TEXT")


#the results of this elevates 1-2" copper and poly pipes around the bulbs of cul de sacs, and other random strange things like a 1" copper waterline under the riverfront plaza building, belle crest drive, 6th & Arizona St,
#selection changed to not include pipes under 2 inch diam
#these should not be scheduled for replacement with 8" pvc, we are targeting more 6" transite, di, CI
# at some point here we need to filter out little short segments, bypasses, tower connections, but keeping stuff like the 1916 4" ci on 7th between Ohio and Tennesssee




<Result 'wMain221012'>

In [28]:
#calculates replacement cost differential, identifying assumed extra value of replaced sub 8" pipe with 8" pipe
arcpy.management.AddField(watermains, "ReplacementDif", "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)
arcpy.management.CalculateField(watermains, "ReplacementDif", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "DIAMETER < 8 AND Shape_Length > 250", None)
arcpy.management.CalculateField(watermains, "ReplacementDif", "(8-!DIAMETER!)", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'wMain221012'>

In [29]:
arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "CLEAR_SELECTION", '', None)
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)

arcpy.management.AddField(watermains, "PriorityScore", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(watermains, "PriorityScore", "(100-!PercentLifeRemaining!)/10+!LOB_ADJ!+!LOB!", "PYTHON3", '', "TEXT")




<Result 'wMain221012'>

In [30]:
#these commands will clear selected features, these commands should normally be the last step run in each cell
arcpy.management.SelectLayerByAttribute(watermains+"_Break_Stats", "CLEAR_SELECTION", '', None)


<Result 'wMain221012_Break_Stats'>

In [31]:
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)

<Result 'wMain221012'>

In [32]:
#Break Threshold Calculation
#break estimate 

BreakEstimateCost = 7000
arcpy.management.AddField(watermains, "BreakRateThreshold", "Double", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(watermains, "BreakRateThreshold", "(math.log(1+0.05))/(math.log(1+7000/!ReplacementCost!))", "PYTHON3", '', "TEXT")



<Result 'wMain221012'>

In [ ]:
arcpy.management.AddField(watermains, "AnnualBreakRate", "Double", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(watermains, "AnnualBreakRate", "!BreakCount!/10", "PYTHON3", '', "TEXT")



In [15]:
arcpy.management.AddField(watermains, "BreakThresholdMet", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "AnnualBreakRate >= BreakRateThreshold", None)
arcpy.management.CalculateField(watermains, "BreakThresholdMet", "1", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)

<Result 'Water_Main_2022_05_26'>

In [36]:
#hydrant flow test data static pressure grid creator

Hydflowtest = r'https://gisweb.lawrenceks.org/hosting/rest/services/Water/All_Hydrant_Flow_Tests/FeatureServer'

#this service seems to show multiple flow test results for each hydrant including static pressure
#this selects the most recent hydrant flow test from all flow tests
arcpy.management.Sort(r"All Hydrant Flow Tests\All Hydrant Flow Tests", projdb+r"\AllHydrantFlowTests_Sorted", "HY_NUMBER ASCENDING;Test_Date DESCENDING;Test_No DESCENDING;Test_Static ASCENDING", "UR")
arcpy.analysis.PairwiseDissolve("AllHydrantFlowTests_Sorted", projdb+r"\AllHydrantFlowTests_Sorted_P", "HY_NUMBER", "Test_No FIRST;Test_Date FIRST;Test_Date MAX;Test_Static FIRST;Test_Static COUNT;Test_Static MAX", "MULTI_PART")

#one hydrant has a static pressure of 801
#another one now has a static pressure greater than 5000


<Result '\\\\citydata\\users\\kgonterwitz\\kyledev\\Waterassets20221012.gdb\\AllHydrantFlowTests_Sorted_P'>

In [38]:
#delete rows with conversion errors or invalid values
arcpy.management.SelectLayerByAttribute("AllHydrantFlowTests_Sorted_P", "NEW_SELECTION", "FIRST_Test_Static > 200 OR FIRST_Test_Static is null", None)
arcpy.management.DeleteFeatures("AllHydrantFlowTests_Sorted_P")

<Result 'AllHydrantFlowTests_Sorted_P'>

In [39]:
#if doesnt work in code review GP history and evalaulte, rerun
out_raster = arcpy.sa.NaturalNeighbor("AllHydrantFlowTests_Sorted_P", "FIRST_Test_Static", 100); out_raster.save(projdb+r"\NN_H2")

arcpy.management.CopyRaster(r"NN_H2", projdb+"\waterrpressure", '', None, "3.4e+38", "NONE", "NONE", '', "NONE", "NONE", "GRID", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Raster: Dataset out_raster does not exist or is not supported
Failed to execute (CopyRaster).


In [22]:
#evaluate the pressure gradient from hydrant flow test, assign pressure ranges 1-5 and test sensitivity to LOB

#watermain = 'Water_Main_202110'

#modify this to use Darren's Hydrant Flow Test service

#if doesnt work in code review GP history and evalaulte, rerun

StaticHg = 'waterpressure'

arcpy.sa.Sample(StaticHg, watermains, r"Sample_waterpre1", "NEAREST", "OBJECTID", "CURRENT_SLICE", None, "MEAN", None, None, "ROW_WISE", "FEATURE_CLASS")
    
#this seems to represent the values from the hydrant flow test static pressure grid "waterpressure" from the waterlines, probably other ways to do it

arcpy.management.AddField(watermains, "HG_Score", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.AddField(watermains, "SampleHg", "DOUBLE", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

#scores entered into histogram, most pressures are between 58-85 psi
#range is about 45-120
#60-85 = 0
#>60=1
#85.1-90 = 2
#90.1-95=3
#95.1-105=4
#>105.1 = 5
arcpy.management.AddJoin(watermains, "OBJECTID", "Sample_waterpre1", "OBJECTID", "KEEP_ALL")

arcpy.management.CalculateField(watermains, watermains+".SampleHg", "!Sample_waterpre1.waterpressure!", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

arcpy.management.RemoveJoin(watermains, "Sample_waterpre1")

ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (CalculateField).


In [23]:
#these steps dont work, can figutre out the process and fix it in geoprocessing history
arcpy.management.AddJoin(watermains, "OBJECTID", "Sample_waterpre1", "OBJECTID", "KEEP_ALL")

arcpy.management.CalculateField(watermains, watermains+".SampleHg", "!Sample_waterpre1.waterpressure!", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

arcpy.management.RemoveJoin(watermains, "Sample_waterpre1")


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (CalculateField).


In [24]:
arcpy.management.RemoveJoin(watermains, "Sample_waterpre1")

<Result 'Water_Main_2022_05_26'>

In [17]:
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "SampleHg < 60", None)
arcpy.management.CalculateField(watermains, "HG_Score", "1", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "SampleHg >= 60 And SampleHg < 85", None)
arcpy.management.CalculateField(watermains, "HG_Score", "0", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "SampleHg >= 85 And SampleHg < 90", None)
arcpy.management.CalculateField(watermains, "HG_Score", "2", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "SampleHg >= 90 And SampleHg < 95", None)
arcpy.management.CalculateField(watermains, "HG_Score", "3", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "SampleHg >= 95 And SampleHg < 105", None)
arcpy.management.CalculateField(watermains, "HG_Score", "4", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "SampleHg >= 105", None)
arcpy.management.CalculateField(watermains, "HG_Score", "5", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

<Result 'wMain221012'>

In [19]:
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "SampleHg is null", None)
arcpy.management.CalculateField(watermains, "HG_Score", "0", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")

<Result 'wMain221012'>

In [21]:
arcpy.management.SelectLayerByAttribute(watermains, "CLEAR_SELECTION", '', None)

arcpy.management.CalculateField(watermains, "PriorityScoreHG", "(100-!PercentLifeRemaining!)/10+!LOB_ADJ!+!LOB!+!HG_Score!", "PYTHON3", '', "LONG")




<Result 'wMain221012'>